<a href="https://colab.research.google.com/github/matwirabc/binaproject/blob/aggregate/bina_aggregate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Einführung
Im folgenden Abschnitt werden die Ausganglage beschrieben, sowie die Zielsetzung erörter.

##1.1 Vorgehen
Die Arbeit ist nach dem "Five-Step Approach" nach CPA (Chartered Professional Accountants) gegliedert. Die fünf Schritte sind:


1.   Ziel(e) setzen
2.   Datenerhebung
3.   Analyse
4.   Aufarbeiten
5.   Entscheiden


##1.2 Ausgangslage
Die zuvor bearbeiteten Datensätze stammten aus unterschiedlichen Quellen und standen bislang in keinem Zusammenhang. In diesem Abschnitt sollen die bereinigten Datensätze nun miteinander in Beziehung gesetzt werden. Folgende Datensätze werden dafür genutzt:


*   Fahrzeuge
*   Flugzeuge
*   Bahn
*   Bevölkerung Schweiz


Der Datensatz "2.4 Bevölkerung Schweiz" soll dabei als verbindendes Element genutz werden.

# Schritt 1: Zielsetzung
Folgende Fragen sollen in der Arbeit bearbeitet und beantwortet werden.

**Hauptfrage:**
Wie haben sich die Verkehrsmittel im Verhältnis zum Bevölkerungswachstum in der Schweiz entwickelt – sowohl relativ als auch absolut?

**Unterfrage 1**
Welches Verkehrsmittel verzeichnete im Untersuchungszeitraum das stärkste Wachstum (relativ und absolut)?

**Unterfrage 2**
Welches Verkehrsmittel erlitt während der Corona-Pandemie den stärksten Rückgang (relativ und absolut)?

# Schritt 2: Datenerhebung
Für die Anaylse der Entwicklung und Trends wurden die Datensätze aus den anderen Bereichen dieser Arbeit verwendet. Die Datensätze sind im GIT-Repository abgelegt.

##2.1 Laden und importieren der Libraries
Die Verarbeitung und Darstellung der Daten erfolgt mit den untenstehenden Libarys, welche in einem ersten Schritt importiert werden.


In [ ]:


#Load necessary packages/libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import matplotlib.ticker as mticker
import plotly.express as px
import numpy as np
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from plotly.subplots import make_subplots
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
!pip install dash
!pip install mplcursors



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


##2.2 Laden der Datensets
Die Datensets wurden in verschiedenen Formaten (.csv und .xlsx) zur Verfügung gestellt.